# Problem Statement

A clique in an undirected graph $𝐺=(𝑉,𝐸)$ is a subset of vertices $C \subset V$ such that every pair of vertices in $C$ is connected by an edge.
A maximal clique is a clique that cannot be extended by including any adjacent vertex , i.e., it’s not contained within any larger clique.

# Background


# reorder implementation 

1. Array Visited, containes verticies that where picked to expand the trees ast start 

2. order, maintains a global order of visted to enumerate over 
    - Not visited, not in clique
    - Not visited, in clique
    - visited, not in clique
    - visited, in clique 



1. Start with a vertex, expand Depth first
    - if found a maximal clique, reorder based on tier 
    - if found duplicated, track back and depth first again 
    - if no clique found, reoder by puting vertx at end 
2. add vertex to visited 
3. repeat with first vertex from order
4. stop when all vertices are visited 

We store the cliques in a data structure 
each vertex is mapped to its clique 

![img](pics/G1.jpeg)

![img](pics/order1.jpeg)

![img](/Users/wajid/Desktop/Research/Projects/BK/adaptiveBKAlgo/pics/order2.jpeg)

![img](pics/order3.jpeg)